In [2]:
import numpy as np
import pandas as pd
from random import*

In [2]:
#Problem. Find the final payoffs of a taxi driver who is in place i at t=0.
#Given data is: origin-destination matrices (OD), corresponding to 15 minutes time bins.
#Imagined data is: Reward matrix (or matrices) of the price of a trip from place i to place j.

In [ ]:
#This creates a transition matrix. Paramenter <<case>> can be "real" or
#"imaginary". <<n>> is the quantity of possible places to be (coordinates). 

def Transition_Matrix(case,n):
    
    if case=="real":
        #Importing the origin destination matrices as ODi (i=1,2,3,...)
        OD1 = pd.read_csv('Demand_Jun_04_2011.csv', header=None)
        OD2 = pd.read_csv('Demand_Jun_05_2011.csv', header=None)
        OD3 = pd.read_csv('Demand_Jun_06_2011.csv', header=None)
        OD4 = pd.read_csv('Demand_Jun_07_2011.csv', header=None)
        OD5 = pd.read_csv('Demand_Jun_08_2011.csv', header=None)
        
        #This creates a numpy matrix with the values of the pandas matrices above.
        OD1ValuesRaw = OD1.values
        OD2ValuesRaw = OD2.values
        OD3ValuesRaw = OD3.values
        OD4ValuesRaw = OD4.values
        OD5ValuesRaw = OD5.values
        
        #This deletes the row and column headers
        OD1Values = np.delete(OD1ValuesRaw,0,1)
        OD1Values = np.delete(OD1Values,0,0)
        OD2Values = np.delete(OD2ValuesRaw,0,1)
        OD2Values = np.delete(OD2Values,0,0)
        OD3Values = np.delete(OD3ValuesRaw,0,1)
        OD3Values = np.delete(OD3Values,0,0)
        OD4Values = np.delete(OD4ValuesRaw,0,1)
        OD4Values = np.delete(OD4Values,0,0)
        OD5Values = np.delete(OD5ValuesRaw,0,1)
        OD5Values = np.delete(OD5Values,0,0)
        
        #This adds up the values of the specific day-time OD matrices.
        sum1=np.add(OD1Values, OD2Values)
        sum2=np.add(sum1, OD3Values)
        sum3=np.add(sum2, OD4Values)
        PreMeanMatrix_Int=np.add(sum3, OD5Values)
        
        #This changes the type of the values of the OD matrix.
        PreMeanMatrix_Float=PreMeanMatrix_Int.astype(float)
        
        #This creates a Mean Matrix. This divides the Pre-Mean Matrix by N, the number of original matrices.
        N=5
        MeanMatrix_Float=(1/N)*PreMeanMatrix_Float
        
        #Creating a copy of the previous matrix, a list of the sum of the row elements.
        PreTransitionMatrix=MeanMatrix_Float.copy()
        RowList=[]
        
        #This is the process to transform the Mean Matrix in the Transition Matrix
        for ii in range(0,PreTransitionMatrix.shape[0]):
            sf=np.sum(PreTransitionMatrix[ii,:])
            RowList.append(sf)
            SumRowList=sum(RowList)
            if sf!=0.0:
                PreTransitionMatrix[ii,:]=PreTransitionMatrix[ii,:]/sf
            else:
                PreTransitionMatrix[ii,:]=1.0/8300
        
        TransitionMatrix=np.asmatrix(PreTransitionMatrix)
        
        return TransitionMatrix
        
    #Now I will suppose I have Q transitions, and n quantity of possible 
    #places do be.
    elif case=="imaginary":
        PreTransitionMatrix=np.zeros((n,n))
        #Filling it with random numbers.
        for ii in range(n):
            for jj in range(n):
                PreTransitionMatrix[ii,jj]=random()
        #Normalizing rows.
        for ii in range(n):
            sf=np.sum(PreTransitionMatrix[ii,:])
            PreTransitionMatrix[ii,:]=PreTransitionMatrix[ii,:]/sf
        
        TransitionMatrix = np.asarray(PreTransitionMatrix)
            
        return TransitionMatrix
            

In [4]:
#I invented a random Reward Matrix.
def Reward_Matrix(n):
    RewardMatrix=np.zeros((n,n))
    for ii in range(n):
            for jj in range(n):
                RewardMatrix[ii,jj]=normalvariate(30.0,10.0)
    return RewardMatrix

In [5]:
#Now, I define the initial state vector.
v0Array=np.zeros(8300, dtype=float)
v0Array[0]=1.0
v0 = np.asmatrix(v0Array)
v0


matrix([[1., 0., 0., ..., 0., 0., 0.]])

In [6]:
TM = Transition_Matrix("real",8300)
RM = Reward_Matrix(8300)

/home/santiagortiz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
print(TM)
print(RM)

[[0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]
 [0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]
 [0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]
 ...
 [0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]
 [0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]
 [0.00012048 0.00012048 0.00012048 ... 0.00012048 0.00012048 0.00012048]]
[[35.38741965 51.15890903 37.3510395  ... 25.7338995  23.81370964
  33.01305957]
 [26.12553498 33.14245277 28.99290137 ... 16.88869449 25.51819214
  16.90822586]
 [16.57505477 26.12296497 39.42857092 ... 24.37974895  5.84872788
  18.20304916]
 ...
 [31.8517845  32.93663997 26.97727385 ... 27.61828509 33.44572679
  34.55520493]
 [16.72613984 35.77327227 26.08539488 ... 28.68544912 32.17874022
  30.90317362]
 [12.51192145 35.36813887 30.2419344  ... 31.7864421  53.06788972
  24.60993554]]


In [8]:
#Following the equation: vf = v0*Transition_Matrix("case",n)
vfmatrix = v0*TM
vfarray = np.asarray(vfmatrix)
vfarray

array([[0.00012048, 0.00012048, 0.00012048, ..., 0.00012048, 0.00012048,
        0.00012048]])

In [9]:
#The final reward vector:

#RewardVector = v0*(Transition_Matrix("imaginary",5)*Reward_Matrix(5))
RewardVector = vfarray*RM
print(RewardVector)

#Now I will calculate the mean reward, given the initial state vector v0.
MeanReward = np.sum(RewardVector)/np.shape(RewardVector)[1]
print(MeanReward)

[[0.00426354 0.00616372 0.00450013 ... 0.00310047 0.00286912 0.00397748]
 [0.00314765 0.00399307 0.00349312 ... 0.00203478 0.00307448 0.00203714]
 [0.00199699 0.00314735 0.00475043 ... 0.00293732 0.00070467 0.00219314]
 ...
 [0.00383756 0.00396827 0.00325027 ... 0.0033275  0.00402961 0.00416328]
 [0.0020152  0.00431003 0.00314282 ... 0.00345608 0.00387696 0.00372327]
 [0.00150746 0.00426122 0.00364361 ... 0.00382969 0.00639372 0.00296505]]
29.999618455040775


In [10]:
#For N transitions.
N = 4

from numpy.linalg import matrix_power
RewardVectorN = (v0*(matrix_power(TM,N)))*(N*RM)
print(RewardVectorN)

MeanRewardN = np.sum(RewardVectorN)/np.shape(RewardVectorN)[1]
print(MeanRewardN)

[[120.40161345 117.94395736 120.73418965 ... 119.49970223 119.90102902
  120.54980189]]
119.99754126595981


In [33]:
def Matrix_Reduced(n, divi):
    
    #Importing the origin destination matrices as ODi (i=1,2,3,...)
    OD1 = pd.read_csv('Demand_Jun_04_2011.csv', header=None)
    OD2 = pd.read_csv('Demand_Jun_05_2011.csv', header=None)
    OD3 = pd.read_csv('Demand_Jun_06_2011.csv', header=None)
    OD4 = pd.read_csv('Demand_Jun_07_2011.csv', header=None)
    OD5 = pd.read_csv('Demand_Jun_08_2011.csv', header=None)
    
    #This creates a numpy matrix with the values of the pandas matrices above.
    OD1ValuesRaw = OD1.values
    OD2ValuesRaw = OD2.values
    OD3ValuesRaw = OD3.values    
    OD4ValuesRaw = OD4.values
    OD5ValuesRaw = OD5.values
        
    #This deletes the row and column headers
    OD1Values = np.delete(OD1ValuesRaw,0,1)
    OD1Values = np.delete(OD1Values,0,0)
    OD2Values = np.delete(OD2ValuesRaw,0,1)
    OD2Values = np.delete(OD2Values,0,0)
    OD3Values = np.delete(OD3ValuesRaw,0,1)
    OD3Values = np.delete(OD3Values,0,0)
    OD4Values = np.delete(OD4ValuesRaw,0,1)
    OD4Values = np.delete(OD4Values,0,0)
    OD5Values = np.delete(OD5ValuesRaw,0,1)
    OD5Values = np.delete(OD5Values,0,0)
        
    #This adds up the values of the specific day-time OD matrices.
    sum1=np.add(OD1Values, OD2Values)
    sum2=np.add(sum1, OD3Values)
    sum3=np.add(sum2, OD4Values)
    PreMeanMatrix_Int=np.add(sum3, OD5Values)
        
    #This changes the type of the values of the OD matrix.
    PreMeanMatrix_Float=PreMeanMatrix_Int.astype(float)
        
    #This creates a Mean Matrix. This divides the Pre-Mean Matrix by N, the number of original matrices.
    N=5
    MeanMatrix_Float=(1/N)*PreMeanMatrix_Float
        
    #Now, this takes the big matrix and divides it in a (d,d) matrix. 
        
    d=int(n/divi)
        
    ReducedMatrix=np.zeros((d,d))
        
    for nn in range(d):
        for mm in range(d):
            ReducedMatrix[nn,mm]=np.sum(MeanMatrix_Float[(nn*divi):((nn*divi)+divi),(mm*divi):((mm*divi)+divi)])
        
    
    return ReducedMatrix
        

In [57]:
Reduced_M = Matrix_Reduced(8300, 4150)

/home/santiagortiz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [60]:
print(Reduced_M)

R_M = Reduced_M.copy()

RowList = []

for ii in range(R_M.shape[0]):
    sf=np.sum(R_M[ii,:])
    RowList.append(sf)
    SumRowList=sum(RowList)
    if sf!=0.0:
        R_M[ii,:]=R_M[ii,:]/sf
    else:
        R_M[ii,:]=1.0/2
        
M_Transition=np.asmatrix(R_M)

print(M_Transition)


[[ 642.4  955.4]
 [ 594.  2667. ]]
[[0.40205282 0.59794718]
 [0.18215271 0.81784729]]


In [64]:
#If you were in ii, you had this quantity of trips:

trips_from_ii = []
for ii in range ( np.shape(Reduced_M)[0] ):
    trips_from_ii.append(np.sum(Reduced_M[:,ii]))
    
#And probability P_ii_any to move to some place.
P_ii_any=[]
for ii in range (2):
    P_ii_any.append(trips_from_ii[ii]/np.sum(trips_from_ii))

print(trips_from_ii)
print(P_ii_any)


[1236.3999999999999, 3622.399999999996]
[0.2544661233226313, 0.7455338766773687]


In [65]:
#trips to jj

trips_to_jj = []
for jj in range ( np.shape(Reduced_M)[1] ):
    trips_to_jj.append(np.sum(Reduced_M[jj,:]))
    
#And probability P_any_jj to move to some place.
P_any_jj=[]
for jj in range (2):
    P_any_jj.append(trips_to_jj[jj]/np.sum(trips_to_jj))

print(trips_to_jj)
print(P_any_jj)


[1597.8000000000009, 3260.999999999995]
[0.32884662879723436, 0.6711533712027656]
